In [64]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster

mi target será la variable sales, significa cantidad de ventas en ese dia

Exogenas:

* Dia de la semana
* feriados
* tipo (store)
* cluster (store)

## Tablas

### train

In [65]:
#df_train= pd.read_csv(r'D:\Escritorio\IES21\4to cuatrimestre\PP2\data\train.csv')
df_train= pd.read_csv(r'C:\Users\Usuario\Desktop\lautaro\ies 21\IES21\4to cuatrimestre\PP2\data\train.csv')
df_train.head(1)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0


In [66]:
#Podemos observar que no tenemos datos nulos en todo nuestro DataSet
df_train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [67]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [68]:
#Para facilitar el trabajo con las fechas, pasamos la columna date a tipo datetime
df_train['date']=pd.to_datetime(df_train['date'])
df_train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [69]:
# Tor 5 Stores con mas ventas
df_train.groupby(['store_nbr'])['sales'].sum().reset_index().sort_values('sales',ascending=False).head()

,store_nbr,sales
43,44,6.208755e+07
44,45,5.449801e+07
46,47,5.094831e+07
2,3,5.048191e+07
48,49,4.342010e+07


In [70]:
# Top 5 familys con mas ventas
df_train.groupby(['family'])['sales'].sum().reset_index().sort_values('sales',ascending=False).head()

,family,sales
12,GROCERY I,3.434627e+08
3,BEVERAGES,2.169545e+08
30,PRODUCE,1.227047e+08
7,CLEANING,9.752129e+07
8,DAIRY,6.448771e+07


In [71]:
df_train=df_train.loc[(df_train['store_nbr'].isin([44,45,47,3,49])) &
                      (df_train['family'].isin(['GROCERY I','BEVERAGES','PRODUCE','CLEANING','DAIRY']))]
df_train

,id,date,store_nbr,family,sales,onpromotion
729,729,2013-01-01,3,BEVERAGES,0.000,0
733,733,2013-01-01,3,CLEANING,0.000,0
734,734,2013-01-01,3,DAIRY,0.000,0
738,738,2013-01-01,3,GROCERY I,0.000,0
756,756,2013-01-01,3,PRODUCE,0.000,0
...,...,...,...,...,...,...
3000528,3000528,2017-08-15,49,BEVERAGES,5820.000,14
3000532,3000532,2017-08-15,49,CLEANING,1765.000,16
3000533,3000533,2017-08-15,49,DAIRY,1832.000,25
3000537,3000537,2017-08-15,49,GROCERY I,7001.000,38


### Test (usar para testear)

In [72]:
#df_test= pd.read_csv(r'D:\Escritorio\IES21\4to cuatrimestre\PP2\data\test.csv')
df_test= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/IES21/4to cuatrimestre/PP2/data/test.csv')
df_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


### Tiendas

In [73]:
#df_store= pd.read_csv(r'D:\Escritorio\IES21\4to cuatrimestre\PP2\data\stores.csv')
df_store= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/IES21/4to cuatrimestre/PP2/data/stores.csv')
df_store.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [74]:
# Cantidad de tiendas
df_store['store_nbr'].count()

54

In [75]:
df_store.loc[df_store['type']=='B']

,store_nbr,city,state,type,cluster
8,9,Quito,Pichincha,B,6
10,11,Cayambe,Pichincha,B,6
17,18,Quito,Pichincha,B,16
19,20,Quito,Pichincha,B,6
20,21,Santo Domingo,Santo Domingo de los Tsachilas,B,6
30,31,Babahoyo,Los Rios,B,10
33,34,Guayaquil,Guayas,B,6
38,39,Cuenca,Azuay,B,6


In [76]:
df_store['cluster'].unique()

array([13,  8,  9,  4,  6, 15,  7,  3, 12, 16,  1, 10,  2,  5, 11, 14, 17],
      dtype=int64)

In [77]:
df_store['type'].unique()

array(['D', 'B', 'C', 'E', 'A'], dtype=object)

### Dias Feriados

In [78]:
#df_holidays_events= pd.read_csv(r'D:\Escritorio\IES21\4to cuatrimestre\PP2\data\holidays_events.csv')
df_holidays_events= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/IES21/4to cuatrimestre/PP2/data/holidays_events.csv')
df_holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [79]:
df_holidays_events=df_holidays_events.loc[df_holidays_events['type']=='Holiday']
df_holidays_events['date']=pd.to_datetime(df_holidays_events['date'])
df_holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
338,2017-11-12,Holiday,Local,Ambato,Independencia de Ambato,False
340,2017-12-06,Holiday,Local,Quito,Fundacion de Quito,True
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
344,2017-12-22,Holiday,Local,Salinas,Cantonizacion de Salinas,False


### Precio petroleo

In [80]:
#df_oil= pd.read_csv(r'D:\Escritorio\IES21\4to cuatrimestre\PP2\data\oil.csv')
df_oil= pd.read_csv('c:/Users/Usuario/Desktop/lautaro/ies 21/IES21/4to cuatrimestre/PP2/data/oil.csv')
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [81]:
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [82]:
df_oil['date']=pd.to_datetime(df_oil['date'])

## EDA

### Cant de ventas

##### Por dia

In [83]:
# Agrupo por columna date y famili, sumando sales de las diferentes stores
df_cant_ventas=df_train.groupby(['date','family'])['sales'].sum().reset_index()
# Pongo a date como indice para usar una herramienta en un futuro
df_cant_ventas.index= df_cant_ventas['date']
df_cant_ventas.head()

,date,family,sales
date,,,
2013-01-01,2013-01-01,BEVERAGES,0.0
2013-01-01,2013-01-01,CLEANING,0.0
2013-01-01,2013-01-01,DAIRY,0.0
2013-01-01,2013-01-01,GROCERY I,0.0
2013-01-01,2013-01-01,PRODUCE,0.0


In [84]:
fig = px.line(df_cant_ventas, x=df_cant_ventas.index, y='sales',color='family')
fig.show()

Se puede observar que todos los 1 de enero, indeferente al año, no se realizan casi ventas

##### Por semanas

In [85]:
# Agrupo el DataFrame df_cant_ventas por semana, promediando sus valores
df_cant_ventas_semana=df_cant_ventas.resample('MS').mean()
df_cant_ventas_semana.head()

,sales
date,
2013-01-01,12050.787097
2013-02-01,11897.000000
2013-03-01,13089.993548
2013-04-01,12694.593333
2013-05-01,12567.058065


In [86]:
fig = px.line(df_cant_ventas_semana, x=df_cant_ventas_semana.index, y='sales')
fig.show()

Otro análisis sin considerar la familia y agrupados por semanas, se puede observar que con el paso del tiempo, se fueron realizando mas ventas

### Precio del petroleo a lo largo del tiempo

In [87]:
fig = px.line(df_oil, x='date', y='dcoilwtico')
fig.show()

Se observa que con el tiempo, el precio del peotroleo fue desendiendo, tambien se aprecia que hay varios dias donde no se cuenta con los datos del precio de petroleo, se imputaran en un futuro.

### Relacion cantidad de ventas y precio del petroleo

In [88]:
df_grupo= df_train.groupby(['date'])['sales'].sum().reset_index()
df_grupo.head()

,date,sales
0,2013-01-01,0.0
1,2013-01-02,92221.0
2,2013-01-03,62884.0
3,2013-01-04,64028.0
4,2013-01-05,90385.0


In [89]:
df_grupo= pd.merge(df_grupo,df_oil, on='date')
df_grupo.head()

,date,sales,dcoilwtico
0,2013-01-01,0.0,NaN
1,2013-01-02,92221.0,93.14
2,2013-01-03,62884.0,92.97
3,2013-01-04,64028.0,93.12
4,2013-01-07,60435.0,93.20


In [90]:
df_grupo=df_grupo.fillna(method='bfill')
df_grupo.head()

,date,sales,dcoilwtico
0,2013-01-01,0.0,93.14
1,2013-01-02,92221.0,93.14
2,2013-01-03,62884.0,92.97
3,2013-01-04,64028.0,93.12
4,2013-01-07,60435.0,93.20


In [91]:
x= df_grupo.drop('date',axis=1)
scaler = MinMaxScaler()
scaler.fit(x)
transformado=scaler.transform(x)
dftransf= pd.DataFrame(transformado, columns=['sales','dcoilwtico'])
dftransf['date']=df_grupo[['date']]
dftransf.head()

,sales,dcoilwtico,date
0,0.000000,0.792965,2013-01-01
1,0.299807,0.792965,2013-01-02
2,0.204434,0.790951,2013-01-03
3,0.208153,0.792728,2013-01-04
4,0.196472,0.793675,2013-01-07


In [92]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=dftransf['date'], y=dftransf['sales'], name='Sales', line=dict(color='firebrick', width=2)))

fig.add_trace(go.Scatter(x=dftransf['date'], y=dftransf['dcoilwtico'], name='Oil', line=dict(color='royalblue', width=2)))

fig.update_layout(title='Relacion de la cantidad total de ventas con precio de petroleo',
                   xaxis_title='Date')
fig.show()

se aprecia qeue a medida que el precio del petroleo fue disminuyendo, la cantidad de ventas iban aumentando, esta relacion sucede mientras avanzamos en el tiempo.

## DataSet para entrenar

In [93]:
df_final=pd.merge(df_train,df_oil,on='date',how='left')
df_final=df_final.fillna(method='bfill')
df_final=pd.merge(df_final,df_holidays_events,on='date',how='left')
df_final=df_final.merge(df_store,on='store_nbr')
df_final.loc[(df_final['type_x'].isna()),'type_x'] = 'Common Day'
df_final.drop(['locale','locale_name','description','transferred','city','state'],axis=1,inplace=True)
df_final

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,type_y,cluster
0,729,2013-01-01,3,BEVERAGES,0.000,0,93.14,Holiday,D,8
1,733,2013-01-01,3,CLEANING,0.000,0,93.14,Holiday,D,8
2,734,2013-01-01,3,DAIRY,0.000,0,93.14,Holiday,D,8
3,738,2013-01-01,3,GROCERY I,0.000,0,93.14,Holiday,D,8
4,756,2013-01-01,3,PRODUCE,0.000,0,93.14,Holiday,D,8
...,...,...,...,...,...,...,...,...,...,...
42495,3000528,2017-08-15,49,BEVERAGES,5820.000,14,47.57,Holiday,A,11
42496,3000532,2017-08-15,49,CLEANING,1765.000,16,47.57,Holiday,A,11
42497,3000533,2017-08-15,49,DAIRY,1832.000,25,47.57,Holiday,A,11
42498,3000537,2017-08-15,49,GROCERY I,7001.000,38,47.57,Holiday,A,11


In [94]:
df_final["type_x"] = df_final["type_x"].map({"Holiday": 1, "Common Day": 2})
df_final["type_y"] = df_final["type_y"].map({"A": 1, "B": 2, "C": 3, "D": 4, "E": 5,})
df_final.replace(list(df_final['family'].unique()), list(np.arange(0,len(list(df_final['family'].unique())))),inplace=True)

In [95]:
df_final=df_final.set_index('date')
df_final.head(1)

,id,store_nbr,family,sales,onpromotion,dcoilwtico,type_x,type_y,cluster
date,,,,,,,,,
2013-01-01,729,3,0,0.0,0,93.14,1,4,8


In [96]:
df_final['store_nbr'].unique()

array([ 3, 44, 45, 47, 49], dtype=int64)

In [97]:
from sklearn.model_selection import train_test_split

In [98]:
x=df_final.drop('sales',axis=1)
y= df_final['sales']

In [111]:
modelo1=None
modelo2=None
modelo3=None
modelo4=None
modelo5=None

In [112]:
tuplas=[(0,modelo1),(1,modelo2),(2,modelo3),(3,modelo4),(4,modelo5)]

In [ ]:
for store in df_final['store_nbr'].unique():
    for family, modelo in tuplas:
        

In [133]:

df_score= pd.DataFrame()
for store in df_final['store_nbr'].unique():
    for family,modelos in tuplas:
# Filtrado de Data Frane
        df_pred= df_final.loc[(df_final['store_nbr']==store) & (df_final['family']==family)]
        df_pred.drop(['id','store_nbr','family'],axis=1,inplace=True)
        df_pred=df_pred.resample('D').mean()
        df_pred=df_pred.fillna(0)        
# Generacion de valores x e y
        x=df_pred.drop('sales',axis=1)
        y= df_pred['sales']
# Division en train y test
        steps=15
# Valores de x
        datos_train_x= x[:-steps]
        datos_test_x= x[-steps:]
# Valores de y
        datos_train_y= y[:-steps]
        datos_test_y= y[-steps:]        
#        x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2, random_state=123)

        forecaster = ForecasterAutoreg(
                        regressor = RandomForestRegressor(random_state=123),
                        lags      = 12 # Este valor será remplazado en el grid search
                )

        lags_grid = [60,70,80]

        param_grid = {'n_estimators': [900,1000,1100],
                'max_depth': [35,40,45]}
        modelos = grid_search_forecaster(
                                forecaster         = forecaster,
                                y                  = datos_train_y,
                                exog               = datos_train_x,
                                param_grid         = param_grid,
                                lags_grid          = lags_grid,
                                steps              = steps,
                                refit              = False,
                                metric             = 'mean_squared_error',
                                initial_train_size = int(len(datos_train_x)*0.5),
                                fixed_train_size   = False,
                                return_best        = True,
                                verbose            = False
                        )
        df_score=df_score.append({'store':store,'family':family,'modelo':modelos,'hiperparams':modelos.iloc[1]})

C:\Users\Usuario\AppData\Local\Temp\ipykernel_10360\742726284.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Number of models compared: 27.


loop lags_grid:   0%|                                               | 0/3 [00:04<?, ?it/s]


KeyboardInterrupt: 

## Modelado

### Modelo puro

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 8
             )

forecaster.fit(y=datos_train['sales'], exog=datos_train[['onpromotion','dcoilwtico','type_x','type_y','cluster']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=steps, exog=datos_train[['onpromotion','dcoilwtico','type_x','type_y','cluster']])

In [ ]:
error_mse = mean_squared_error(
                y_true = datos_test['sales'],
                y_pred = predicciones
            )

print(f"Error de test (mse): {error_mse}")

### Hiperparameros

In [ ]:
steps=15
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 12 # Este valor será remplazado en el grid search
             )

lags_grid = [60,70,80]

param_grid = {'n_estimators': [900,1000,1100],
              'max_depth': [35,40,45]}

resultados_grid = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = datos_train['sales'],
                        exog               = datos_train['dcoilwtico'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = steps,
                        refit              = False,
                        metric             = 'mean_squared_error',
                        initial_train_size = int(len(datos_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
                    )
                    

In [ ]:
resultados_grid

In [ ]:
predicciones = forecaster.predict(steps=steps, exog=datos_test[['onpromotion','dcoilwtico','type_x','type_y','cluster']])

In [ ]:
error_mse = mean_squared_error(y_true = datos_test['sales'], y_pred = predicciones)
print(f"Error de test (mse) {error_mse}")

In [ ]:
predicciones

In [ ]:
datos_test['sales']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=datos_test.index, y=datos_test['sales'], name='Test', line=dict(color='firebrick', width=2)))

fig.add_trace(go.Scatter(x=predicciones.index, y=predicciones, name='prediccion', line=dict(color='royalblue', width=2)))

fig.show()